<a href="https://colab.research.google.com/github/JuyeongKime2/Credit-card-fraud-detecting-system/blob/main/Final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **[B주제 : credit card fraud detecting system (신용카드 이상거래 감지 프로그램)]**

* 약 28만건의 신용카드 거래 데이터 중에서 이상거래(Fraud)를 감지해내는 프로그램

- 사용 데이터 : credit card fraud transaction dataset (Kaggle)
- 실습 환경: Google Colab notebooks
- 학습 언어: Python3

# **Module Import**

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler

# **Access Google Drive files**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


# **Data Load**

In [ ]:
#Load a CSV file from Google Drive using pandas
df = pd.read_csv("/content/drive/MyDrive/dataset/creditcard (1).csv", delimiter=',', dtype=np.float32)

#DataFrame의 행과 열의 개수를 튜플 형태로 반환 후 출력
print(df.shape)

#DataFrame의 상위 5개 행을 출력
df.head()

# **Missing Value Check**



* Index Range: DataFrame의 인덱스 범위.
* Column Names: 각 열의 이름.
* Non-Null Count: 각 열에서 결측값이 아닌 non-null 값의 개수.
* Dtype: 각 열의 데이터 유형(data type).
* Memory Usage: DataFrame이 사용하는 메모리 양.



In [ ]:
#1차 검수 : df의 요약 정보를 출력
df.info()

# 결측값 존재 여부확인

"""
df.isnull().sum().sum() : DataFrame 전체에서 결측값의 총 개수를 계산
결측값이 하나도 없으면 "결측값이 존재하지 않음"을 출력하고,
결측값이 하나라도 있으면 "결측값이 존재함"을 출력

"""
if df.isnull().sum().sum() == 0:
    print("결측값이 존재하지 않음")
else:
    print("결측값이 존재함")

In [ ]:
#2차 검수 : 각 변수에 대해 결측값 존재여부 확인
df.isnull().sum() / df.shape[0]

# **Correlation Visualize(상관관계 시각화)**


**상관계수 (Correlation Coefficient):**\
 -1에서 1 사이의 값을 가지며, 두 변수 간의 선형 관계를 나타낸다.

* 1: 완벽한 양의 상관관계 (두 변수가 함께 증가)
* -1: 완벽한 음의 상관관계 (한 변수가 증가할 때 다른 변수는 감소)
* 0: 상관관계 없음 (두 변수 간에 선형 관계가 없음)

In [ ]:
import seaborn as sns

# Seaborn 스타일 설정
sns.set(style="whitegrid")

# 그래프 크기 설정
f, ax = plt.subplots(figsize=(25, 15))

# 상관 행렬 계산
corr = df.corr()

# 상관 행렬에서 결측값을 0으로 대체 (필요 시)
corr = corr.fillna(0)

# 히트맵 그리기
heatmap = sns.heatmap(corr, annot=True, linewidths=0.3, fmt=".2f", ax=ax, cmap="viridis", cbar_kws={"shrink": 0.5})

# 그래프 제목 추가
plt.title('Correlation Heatmap', fontsize=20)


"""
가독성을 개선하기 위한 옵션 추가

"""

# x축과 y축 레이블 회전
heatmap.set_xticklabels(heatmap.get_xticklabels(), rotation=45, horizontalalignment='right')
heatmap.set_yticklabels(heatmap.get_yticklabels(), rotation=0)

# 색상 바 추가
cbar = heatmap.collections[0].colorbar
cbar.set_label('Correlation Coefficient', fontsize=15)

# 상관계수 조건부 강조 (절대값이 0.8 이상인 경우 붉은색으로 강조)
for i in range(len(corr.columns)):
    for j in range(len(corr.columns)):
        if abs(corr.iloc[i, j]) >= 0.8:
            heatmap.add_patch(plt.Rectangle((j, i), 1, 1, fill=False, edgecolor='r', lw=3))

# 그래프 표시
plt.show()



*   결과적으로, Feature들 사이의 상관관계가 매우 낮게 나옴

* 즉, 상관계수가 0.8 이상이 아니므로 다중공선성의 가능성이 낮기 때문에 다중공선성을 줄일 필요는 없다.



# EDA (Exploaratory Data Analysis, 탐색적 데이터 분석)

데이터의 전체적인 구조 파악

In [ ]:
# "Class" 열의 값 빈도 계산
class_counts = df["Class"].value_counts()
class_ratios = df["Class"].value_counts(normalize=True)

# 클래스 수
print("Class Counts:\n", class_counts)

#클래스 비율
print("\nClass Ratios:\n", class_ratios)

* 0.0 : 정상거래 / 1.0 : 이상거래
* 정상거래는 28만 4315건, 이상거래는 492건이 존재
* 전체 데이터셋에서 오직 0.0017%가 이상거래이므로 심각한 imbalance(불균형)가 존재하는 dataset임을 파악할 수 있음.


* 일반적인 상황에서 주로 사용되는 Accuracy(정확도)를 사용해서 모델의 Performance(성능)를 측정하기 어렵다는 결론이 도출

## **Precision, Recall, F1 Score를 사용한 모델설계**

* Precision(정밀도): 모델이 True라고 분류한 것 중에서 실제 True가 차지하는 비중

* Recall(재현률): 실제 True 중에서 모델이 True라고 분류한 데이터의 비중

* F1 score: Precision과 Recall의 조화평균


In [ ]:
"""
Data imabalance가 얼마나 심한지 시각화를 통해서 확인

"""

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score


# 특징 (X)과 레이블 (y) 분리
X = df.drop('Class', axis=1)  # 'class' 열을 제외한 모든 열을 특징으로 사용
y = df['Class']

# 훈련 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 학습 (예시: 로지스틱 회귀)
model = LogisticRegression()
model.fit(X_train, y_train)

# 테스트 데이터 예측
y_pred = model.predict(X_test)

# Precision, Recall, F1 Score 계산
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

#Visualize
labels = ["Normal", "Fraud"]

# count_classes가 각 클래스의 개수를 담고 있는 Series라고 가정
class_counts.plot(kind="bar")
plt.xticks(range(2), labels, rotation=0)  # x축 레이블을 0도(수평)로 회전
plt.title("Transaction Class Distribution")
plt.xlabel("Class")
plt.ylabel("Count")
plt.show()

\
[현재 모델의 성능 지표]\
\
정밀도: 61.11%\
재현율: 56.12%\
F1 스코어: 58.51%


\
결과: 모델이 사기 거래를 예측하는 데 있어서 어느 정도의 성능을 보여주고 있지만, 여전히 개선의 여지가 있음을 나타냄. 특히 재현율이 낮기 때문에 실제 사기 거래를 많이 놓치고 있을 가능성이 있음.

Data imabalance(데이터 불균형) 가 얼마나 심한지 시각화를 통해서 확인

In [ ]:
df.drop(columns = "Time", inplace=True)
df.head()

Time이라는 Feature는 단순히 신용카드 거래가 이루어진 순서를
기록한 것이기 때문에 분석에 크게 도움이 되지 않으므로 데이터 테이블에서 배제

In [ ]:
# x_data와 y_data를 numpy 배열로 변환하고 데이터 타입을 float32로 설정
x_data = df.iloc[:, :-1].to_numpy(dtype=np.float32)
y_data = df.iloc[:, -1].to_numpy(dtype=np.float32).reshape(-1, 1)
"""
가독성을 높이기 위한 코드:

to_numpy() 메서드를 사용하여 DataFrame을 numpy 배열로 변환하고,
dtype 인자를 사용하여 데이터 타입을 지정.
reshape(-1, 1)을 사용하여 y_data를 2차원 배열로 변환

"""

# 배열의 형태 출력
print(x_data.shape, y_data.shape)





Feature들로 이루어진 x_data와, label을 나타내는 y_data로 원본 데이터를 분리. 분리된 데이터는 앞으로 계산을 해주어야 하기 때문에 동일한 타입의 실수형 데이터로 저장




# **Data Preprocessing - Normalize (데이터전처리-정규화)**

데이터의 Scale에 따라서 결과가 왜곡되는 것을 방지하기 위해서 MinMaxScaler를 통해서 데이터가 0~1 사이의 값을 지니도록 조정해줌

In [ ]:
# MinMaxScaler 객체 생성
scaler = MinMaxScaler()
# 데이터 스케일링
x_data = pd.DataFrame(scaler.fit_transform(x_data))
"""
MinMaxScaler는 fit_transform 메서드를 사용하여 데이터를 0과 1 사이로 변환
결과적으로 각 특성의 최소값은 0, 최대값은 1

"""
# 스케일된 데이터 출력
print(f"스케일된 데이터:\n\n", x_data.head)

* 랜덤 포레스트 모델은 트리 기반 모델로, 각 특성의 스케일에 크게 영향을 받지 않아 정규화가 필수적이지 않지만, 데이터 전처리 과정에서 이상치(outlier)를 줄이고 모델의 안정성을 높이기 위해 정규화를 수행할 수 있음

## **RandomForest(랜덤 포레스트)**

랜덤 포레스트는 결정 트리 기반의 앙상블 모델로, 경사 하강법을 사용하지 않기 때문에 learning rate와 optimizer를 설정할 필요가 없음.\
대신, 랜덤 포레스트 모델을 구현할 때는 다음과 같은 하이퍼파라미터들을 조정할 수 있다.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 랜덤 포레스트 모델 생성
model = RandomForestClassifier(n_estimators=100, random_state=42)  # n_estimators는 트리 개수

# 모델 학습
model.fit(X_train, y_train)

# 테스트 데이터 예측
y_pred = model.predict(X_test)

# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

In [ ]:
model

[성능 지표 해석]

* Accuracy: 0.9996
해석: 모델이 전체 데이터 중 약 99.96%를 올바르게 예측

* Precision: 0.9740
해석: 모델이 사기 거래라고 예측한 것 중 약 97.40%가 실제로 사기 거래\
높은 정밀도: False Positive가 적다는 것을 의미. 즉, 잘못된 양성 예측이 비교적 적음

* Recall: 0.7653
해석: 실제 사기 거래 중 약 76.53%를 모델이 정확히 예측.\
낮은 재현율: False Negative가 많다는 것을 의미. 즉, 실제로 사기 거래인 경우를 모델이 놓치는 경우가 많음.

* F1 Score: 0.8571
해석: 모델이 사기 거래를 예측하는 데 있어서 정밀도와 재현율의 균형이 약 85.71%\
균형 지표: 정밀도와 재현율이 모두 중요할 때 유용한 지표

결과적으로 랜덤포레스트 모델로 F1 score가 높게 나온 것을 알 수 있음.

그러나,
높은 정밀도를 가지고 있지만,재현율이 낮아 실제 사기 거래를 많이 놓치고 있다. 성능향상이 필요할 듯함

## **Threshold Adjustment(임계값 조정)**

모델의 예측 확률에 대해 임계값(Threshold)을 조정하여 재현율을 높일 수 있음. 예를들어, 기본적으로 0.5로 설정된 임계값을 낮추면 더 많은 거래를 사기 거래로 분류하게 되어 재현율이 높아질 수 있음

In [ ]:
!pip install scikit-learn

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve, precision_score, recall_score, accuracy_score, f1_score
import numpy as np

# 랜덤 포레스트 모델 생성 (n_estimators는 트리 개수)
model = RandomForestClassifier(n_estimators=100, random_state=42)

# 모델 학습
model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측 확률 계산
y_scores = model.predict_proba(X_test)[:, 1]

# 정밀도-재현율 곡선 계산
precision, recall, thresholds = precision_recall_curve(y_test, y_scores)

# F1 스코어 계산 및 최적 임계값 찾기
f1_scores = 2 * precision * recall / (precision + recall)
optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_idx]

# 최적 임계값을 사용하여 예측값 생성
y_pred = (y_scores >= optimal_threshold).astype(int)

# 새로운 정밀도, 재현율, 정확도, F1 점수 계산
new_precision = precision_score(y_test, y_pred)
new_recall = recall_score(y_test, y_pred)
new_accuracy = accuracy_score(y_test, y_pred)
new_f1 = f1_score(y_test, y_pred)

# 결과 출력
print(f"Optimal Threshold: {optimal_threshold}")
print(f"New Precision: {new_precision}")
print(f"New Recall: {new_recall}")
print(f"New Accuracy: {new_accuracy}")
print(f"New F1 Score: {new_f1}")

[성능 지표 분석]

Precision (정밀도):
 정밀도가 0.9518로 매우 높음. 이는 모델이 긍정 예측을 할 때 대부분 정확

Recall (재현율):
재현율이 0.8061로 상당히 높음. 모델이 실제 긍정 클래스의 대부분을 올바르게 예측

Accuracy (정확도):
정확도가 0.9996로 매우 높음. 모델이 대부분의 예측을 정확하게 하고있음

F1 Score:
F1 점수가 0.8729로 매우 높음. 이는 모델의 전반적인 성능이 매우 좋다는 것을 의미

결론:

이 모델은 정밀도, 재현율, 정확도, F1 점수 모두 매우 높은 수준.
 이는 모델이 긍정 클래스를 잘 예측할 뿐만 아니라, 전체적으로도 매우 정확하게 예측하고 있다는 것을 의미한다.

다만, 재현율을 조금 더 높일 수 있다면 더 완벽한 모델이 될 수 있을 것이므로 모델의 하이퍼파라미터 튜닝을 고려해봐도 좋음.